In [4]:
import cadquery as cq
from jupyter_cadquery.cadquery import Assembly, Part, Edges, Faces, show

from jupyter_cadquery import patch_cq
patch_cq()

from sidecar import Sidecar
from jupyter_cadquery.cad_objects import set_sidecar

sc = Sidecar(title="CadQuery")
set_sidecar(sc)

from jupyter_cadquery import Replay
replay = Replay(sc, debug=False).replay

Patch
  - cadquery.Workplane.lineTo: guard against line length 0
  - cadquery.Workplane: remove auto display
  - cadquery.Shape: remove auto display
Plugging into cadquery.Workplane to enable replay


## Ex001 Simple Block

In [2]:
# These can be modified rather than hardcoding values for each dimension.
length = 80.0       # Length of the block
height = 60.0       # Height of the block
thickness = 10.0    # Thickness of the block

# Create a 3D block based on the dimension variables above.
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the X and Y origins to define the workplane, meaning that the
# positive Z direction is "up", and the negative Z direction is "down".
result = cq.Workplane("XY").box(length, height, thickness)

replay(result)

Done, using side car 'CadQuery'


## Ex002 Block With Bored Center Hole

In [2]:
# These can be modified rather than hardcoding values for each dimension.
length = 80.0               # Length of the block
height = 60.0               # Height of the block
thickness = 10.0            # Thickness of the block
center_hole_dia = 22.0      # Diameter of center hole in block

# Create a block based on the dimensions above and add a 22mm center hole.
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the X and Y origins to define the workplane, meaning that the
# positive Z direction is "up", and the negative Z direction is "down".
# 2.  The highest (max) Z face is selected and a new workplane is created on it.
# 3.  The new workplane is used to drill a hole through the block.
# 3a. The hole is automatically centered in the workplane.
result = cq.Workplane("XY").box(length, height, thickness) \
    .faces(">Z").workplane().hole(center_hole_dia)

replay(result)

Done, using side car 'CadQuery'


## Ex003 Pillow Block With Counterbored Holes

In [3]:
# These can be modified rather than hardcoding values for each dimension.
length = 80.0                               # Length of the block
width = 60.0                                # Width of the block
height = 60.0               # Height of the block
thickness = 10.0                         # Thickness of the block
center_hole_dia = 22.0             # Diameter of center hole in block
cbore_hole_diameter = 2.4     # Bolt shank/threads clearance hole diameter
cbore_inset = 12.0                    # How far from the edge the cbored holes are set
cbore_diameter = 4.4              # Bolt head pocket hole diameter
cbore_depth = 2.1                   # Bolt head pocket hole depth

# Create a 3D block based on the dimensions above and add a 22mm center hold
# and 4 counterbored holes for bolts
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the X and Y origins to define the workplane, meaning that the
# positive Z direction is "up", and the negative Z direction is "down".
# 2.  The highest(max) Z face is selected and a new workplane is created on it.
# 3.  The new workplane is used to drill a hole through the block.
# 3a. The hole is automatically centered in the workplane.
# 4.  The highest(max) Z face is selected and a new workplane is created on it.
# 5.  A for-construction rectangle is created on the workplane based on the
#     block's overall dimensions.
# 5a. For-construction objects are used only to place other geometry, they
#     do not show up in the final displayed geometry.
# 6.  The vertices of the rectangle (corners) are selected, and a counter-bored
#     hole is placed at each of the vertices (all 4 of them at once).
result = cq.Workplane("XY").box(length, height, thickness) \
    .faces(">Z").workplane().hole(center_hole_dia) \
    .faces(">Z").workplane() \
    .rect(length - cbore_inset, height - cbore_inset, forConstruction=True) \
    .vertices().cboreHole(cbore_hole_diameter, cbore_diameter, cbore_depth) \
    .edges("|Z").fillet(2.0)

replay(result)

Done, using side car 'CadQuery'


## Ex004 Extruded Cylindrical Plate

In [8]:
# These can be modified rather than hardcoding values for each dimension.
circle_radius = 50.0        # Radius of the plate
thickness = 13.0            # Thickness of the plate
rectangle_width = 13.0      # Width of rectangular hole in cylindrical plate
rectangle_length = 19.0     # Length of rectangular hole in cylindrical plate

# Extrude a cylindrical plate with a rectangular hole in the middle of it.
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the named plane orientation "front" to define the workplane, meaning
#     that the positive Z direction is "up", and the negative Z direction
#     is "down".
# 2.  The 2D geometry for the outer circle is created at the same time as the
#     rectangle that will create the hole in the center.
# 2a. The circle and the rectangle will be automatically centered on the
#     workplane.
# 2b. Unlike some other functions like the hole(), circle() takes
#     a radius and not a diameter.
# 3.  The circle and rectangle are extruded together, creating a cylindrical
#     plate with a rectangular hole in the center.
# 3a. circle() and rect() could be changed to any other shape to completely
#     change the resulting plate and/or the hole in it.
result = cq.Workplane("front").circle(circle_radius) \
                              .rect(rectangle_width, rectangle_length) \
                              .extrude(thickness)

replay(result)

Done, using side car 'CadQuery'


## Ex005 Extruded Lines and Arcs

In [9]:
# These can be modified rather than hardcoding values for each dimension.
width = 2.0         # Overall width of the plate
thickness = 0.25    # Thickness of the plate

# Extrude a plate outline made of lines and an arc
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the named plane orientation "front" to define the workplane, meaning
#     that the positive Z direction is "up", and the negative Z direction
#     is "down".
# 2.  Draws a line from the origin to an X position of the plate's width.
# 2a. The starting point of a 2D drawing like this will be at the center of the
#     workplane (0, 0) unless the moveTo() function moves the starting point.
# 3.  A line is drawn from the last position straight up in the Y direction
#     1.0 millimeters.
# 4.  An arc is drawn from the last point, through point (1.0, 1.5) which is
#     half-way back to the origin in the X direction and 0.5 mm above where
#     the last line ended at. The arc then ends at (0.0, 1.0), which is 1.0 mm
#     above (in the Y direction) where our first line started from.
# 5.  An arc is drawn from the last point that ends on (-0.5, 1.0), the sag of
#     the curve 0.2 determines that the curve is concave with the midpoint 0.1 mm
#     from the arc baseline. If the sag was -0.2 the arc would be convex.
#     This convention is valid when the profile is drawn counterclockwise.
#     The reverse is true if the profile is drawn clockwise.
#     Clockwise:        +sag => convex,  -sag => concave
#     Counterclockwise: +sag => concave, -sag => convex
# 6.  An arc is drawn from the last point that ends on (-0.7, -0.2), the arc is
#     determined by the radius of -1.5 mm.
#     Clockwise:        +radius => convex,  -radius => concave
#     Counterclockwise: +radius => concave, -radius => convex
# 7.  close() is called to automatically draw the last line for us and close
#     the sketch so that it can be extruded.
# 7a. Without the close(), the 2D sketch will be left open and the extrude
#     operation will provide unpredictable results.
# 8.  The 2D sketch is extruded into a solid object of the specified thickness.
result = cq.Workplane("front").lineTo(width, 0) \
                              .lineTo(width, 1.0) \
                              .threePointArc((1.0, 1.5), (0.0, 1.0)) \
                              .sagittaArc((-0.5, 1.0), 0.2) \
                              .radiusArc((-0.7, -0.2), -1.5) \
                              .close().extrude(thickness)

replay(result)

Done, using side car 'CadQuery'


## Ex006 Moving the Current Working Point

In [11]:
# These can be modified rather than hardcoding values for each dimension.
circle_radius = 3.0     # The outside radius of the plate
thickness = 0.25        # The thickness of the plate

# Make a plate with two cutouts in it by moving the workplane center point
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the named plane orientation "front" to define the workplane, meaning
#     that the positive Z direction is "up", and the negative Z direction
#     is "down".
# 1b. The initial workplane center point is the center of the circle, at (0,0).
# 2.  A circle is created at the center of the workplane
# 2a. Notice that circle() takes a radius and not a diameter
result = cq.Workplane("front").circle(circle_radius)

# 3.  The work center is movide to (1.5, 0.0) by calling center().
# 3a. The new center is specified relative to the previous center,not
#     relative to global coordinates.
# 4.  A 0.5mm x 0.5mm 2D square is drawn inside the circle.
# 4a. The plate has not been extruded yet, only 2D geometry is being created.
result = result.center(1.5, 0.0).rect(0.5, 0.5)

# 5.  The work center is moved again, this time to (-1.5, 1.5).
# 6.  A 2D circle is created at that new center with a radius of 0.25mm.
result = result.center(-1.5, 1.5).circle(0.25)

# 7.  All 2D geometry is extruded to the specified thickness of the plate.
# 7a. The small circle and the square are enclosed in the outer circle of the
#      plate and so it is assumed that we want them to be cut out of the plate.
#      A separate cut operation is not needed.
result = result.extrude(thickness)

replay(result)

Done, using side car 'CadQuery'


## Ex007 Using Point Lists

In [5]:
# These can be modified rather than hardcoding values for each dimension.
plate_radius = 2.0          # The radius of the plate that will be extruded
hole_pattern_radius = 0.25  # Radius of circle where the holes will be placed
thickness = 0.125           # The thickness of the plate that will be extruded

# Make a plate with 4 holes in it at various points in a polar arrangement from
# the center of the workplane.
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the named plane orientation "front" to define the workplane, meaning
#     that the positive Z direction is "up", and the negative Z direction
#     is "down".
# 2.  A 2D circle is drawn that will become though outer profile of the plate.
r = cq.Workplane("front").circle(plate_radius)

# 3. Push 4 points on the stack that will be used as the center points of the
#    holes.
r = r.pushPoints([(1.5, 0), (0, 1.5), (-1.5, 0), (0, -1.5)])

# 4. This circle() call will operate on all four points, putting a circle at
#    each one.
r = r.circle(hole_pattern_radius)

# 5.  All 2D geometry is extruded to the specified thickness of the plate.
# 5a. The small hole circles are enclosed in the outer circle of the plate and
#     so it is assumed that we want them to be cut out of the plate.  A
#     separate cut operation is not needed.
result = r.extrude(thickness)

replay(result)

Done, using side car 'CadQuery'


## Ex008 Polygon Creation

In [6]:
# These can be modified rather than hardcoding values for each dimension.
width = 3.0         # The width of the plate
height = 4.0        # The height of the plate
thickness = 0.25    # The thickness of the plate
polygon_sides = 6   # The number of sides that the polygonal holes should have
polygon_dia = 1.0   # The diameter of the circle enclosing the polygon points

# Create a plate with two polygons cut through it
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the named plane orientation "front" to define the workplane, meaning
#     that the positive Z direction is "up", and the negative Z direction
#     is "down".
# 2.  A 3D box is created in one box() operation to represent the plate.
# 2a. The box is centered around the origin, which creates a result that may
#     be unituitive when the polygon cuts are made.
# 3.  2 points are pushed onto the stack and will be used as centers for the
#     polygonal holes.
# 4.  The two polygons are created, on for each point, with one call to
#     polygon() using the number of sides and the circle that bounds the
#     polygon.
# 5.  The polygons are cut thru all objects that are in the line of extrusion.
# 5a. A face was not selected, and so the polygons are created on the
#     workplane. Since the box was centered around the origin, the polygons end
#     up being in the center of the box. This makes them cut from the center to
#     the outside along the normal (positive direction).
# 6.  The polygons are cut through all objects, starting at the center of the
#     box/plate and going "downward" (opposite of normal) direction. Functions
#     like cutBlind() assume a positive cut direction, but cutThruAll() assumes
#     instead that the cut is made from a max direction and cuts downward from
#     that max through all objects.
result = cq.Workplane("front").box(width, height, thickness) \
                                    .pushPoints([(0, 0.75), (0, -0.75)]) \
                                    .polygon(polygon_sides, polygon_dia) \
                                    .cutThruAll()

replay(result)

Done, using side car 'CadQuery'


## Ex009 Polylines

In [7]:
# These can be modified rather than hardcoding values for each dimension.
# Define up our Length, Height, Width, and thickness of the beam
(L, H, W, t) = (100.0, 20.0, 20.0, 1.0)

# Define the points that the polyline will be drawn to/thru
pts = [
    (W/2.0, H/2.0),
    (W/2.0, (H/2.0 - t)),
    (t/2.0, (H/2.0-t)),
    (t/2.0, (t - H/2.0)),
    (W/2.0, (t - H/2.0)),
    (W/2.0, H/-2.0),
    (0, H/-2.0)
]

# We generate half of the I-beam outline and then mirror it to create the full
# I-beam.
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the named plane orientation "front" to define the workplane, meaning
#     that the positive Z direction is "up", and the negative Z direction
#     is "down".
# 2.  moveTo() is used to move the first point from the origin (0, 0) to
#     (0, 10.0), with 10.0 being half the height (H/2.0). If this is not done
#     the first line will start from the origin, creating an extra segment that
#     will cause the extrude to have an invalid shape.
# 3.  The polyline function takes a list of points and generates the lines
#     through all the points at once.
# 3.  Only half of the I-beam profile has been drawn so far. That half is
#     mirrored around the Y-axis to create the complete I-beam profile.
# 4.  The I-beam profile is extruded to the final length of the beam.
result = cq.Workplane("front").moveTo(0, H/2.0) \
                              .polyline(pts) \
                              .mirrorY() \
                              .extrude(L)

replay(result)

Done, using side car 'CadQuery'


## Ex010 Defining an Edge with a Spline

In [15]:
# 1.  Establishes a workplane to create the spline on to extrude.
# 1a. Uses the X and Y origins to define the workplane, meaning that the
# positive Z direction is "up", and the negative Z direction is "down".
s = cq.Workplane("XY")

# The points that the spline will pass through
sPnts = [
    (2.75, 1.5),
    (2.5, 1.75),
    (2.0, 1.5),
    (1.5, 1.0),
    (1.0, 1.25),
    (0.5, 1.0),
    (0, 1.0)
]

# 2.  Generate our plate with the spline feature and make sure it is a
#     closed entity
r = s.lineTo(3.0, 0).lineTo(3.0, 1.0).spline(sPnts).close()

# 3.  Extrude to turn the wire into a plate
result = r.extrude(0.5)

replay(result)

Done, using side car 'CadQuery'


## Ex011 Mirroring Symmetric Geometry

In [16]:
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the named plane orientation "front" to define the workplane, meaning
#     that the positive Z direction is "up", and the negative Z direction
#     is "down".
# 2.  A horizontal line is drawn on the workplane with the hLine function.
# 2a. 1.0 is the distance, not coordinate. hLineTo allows using xCoordinate
#     not distance.
r = cq.Workplane("front").hLine(1.0)

# 3.  Draw a series of vertical and horizontal lines with the vLine and hLine
#     functions.
r = r.vLine(0.5).hLine(-0.25).vLine(-0.25).hLineTo(0.0)

# 4.  Mirror the geometry about the Y axis and extrude it into a 3D object.
result = r.mirrorY().extrude(0.25)

replay(result)

Done, using side car 'CadQuery'


## Ex012 Creating Workplanes on Faces

In [17]:
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the named plane orientation "front" to define the workplane, meaning
#     that the positive Z direction is "up", and the negative Z direction
#     is "down".
# 2.  Creates a 3D box that will have a hole placed in it later.
result = cq.Workplane("front").box(2, 3, 0.5)

# 3.  Find the top-most face with the >Z max selector.
# 3a. Establish a new workplane to build geometry on.
# 3b.  Create a hole down into the box.
result = result.faces(">Z").workplane().hole(0.5)

replay(result)

Done, using side car 'CadQuery'


## Ex013 Locating a Workplane on a Vertex

In [ ]:
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the named plane orientation "front" to define the workplane, meaning
#     that the positive Z direction is "up", and the negative Z direction
#     is "down".
# 2.  Creates a 3D box that will have a hole placed in it later.
result = cq.Workplane("front").box(3, 2, 0.5)

# 3.  Select the lower left vertex and make a workplane.
# 3a. The top-most Z face is selected using the >Z selector.
# 3b. The lower-left vertex of the faces is selected with the <XY selector.
# 3c. A new workplane is created on the vertex to build future geometry on.
result = result.faces(">Z").vertices("<XY").workplane()

# 4.  A circle is drawn with the selected vertex as its center.
# 4a. The circle is cut down through the box to cut the corner out.
result = result.circle(1.0).cutThruAll()

replay(result)

## Ex014 Offset Workplanes

In [3]:
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the named plane orientation "front" to define the workplane, meaning
#     that the positive Z direction is "up", and the negative Z direction
#     is "down".
# 2.  Creates a 3D box that will have geometry based off it later.
result = cq.Workplane("front").box(3, 2, 0.5)

# 3.  The lowest face in the X direction is selected with the <X selector.
# 4. A new wokrplane is created
# 4a.The workplane is offset from the object surface so that it is not touching
#    the original box.
result = result.faces("<X").workplane(offset=0.75)

# 5. Creates a thin disc on the offset workplane that is floating near the box.
result = result.circle(1.0).extrude(0.5)

replay(result)

Done, using side car 'CadQuery'


## Ex015 Rotated Workplanes

In [19]:
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the named plane orientation "front" to define the workplane, meaning
#     that the positive Z direction is "up", and the negative Z direction
#     is "down".
# 2.  Creates a plain box to base future geometry on with the box() function.
# 3.  Selects the top-most Z face of the box.
# 4.  Creates a new workplane and then moves and rotates it with the
#     transformed function.
# 5.  Creates a for-construction rectangle that only exists to use for placing
#     other geometry.
# 6.  Selects the vertices of the for-construction rectangle.
# 7.  Places holes at the center of each selected vertex.
# 7a. Since the workplane is rotated, this results in angled holes in the face.
result = cq.Workplane("front").box(4.0, 4.0, 0.25).faces(">Z") \
                 .workplane() \
                 .transformed(offset=(0, -1.5, 1.0), rotate=(60, 0, 0)) \
                 .rect(1.5, 1.5, forConstruction=True).vertices().hole(0.25)

replay(result)

Done, using side car 'CadQuery'


## Ex016 Using Construction Geometry

In [20]:
# Create a block with holes in each corner of a rectangle on that workplane.
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the named plane orientation "front" to define the workplane, meaning
#     that the positive Z direction is "up", and the negative Z direction
#     is "down".
# 2.  Creates a plain box to base future geometry on with the box() function.
# 3.  Selects the top-most Z face of the box.
# 4.  Creates a new workplane to build new geometry on.
# 5.  Creates a for-construction rectangle that only exists to use for placing
#     other geometry.
# 6.  Selects the vertices of the for-construction rectangle.
# 7.  Places holes at the center of each selected vertex.
result = cq.Workplane("front").box(2, 2, 0.5)\
                 .faces(">Z").workplane() \
                 .rect(1.5, 1.5, forConstruction=True).vertices() \
                 .hole(0.125)

replay(result)

Done, using side car 'CadQuery'


## Ex017 Shelling to Create Thin Features

In [21]:
# Create a hollow box that's open on both ends with a thin wall.
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the named plane orientation "front" to define the workplane, meaning
#     that the positive Z direction is "up", and the negative Z direction
#     is "down".
# 2.  Creates a plain box to base future geometry on with the box() function.
# 3.  Selects faces with normal in +z direction.
# 4.  Create a shell by cutting out the top-most Z face.
result = cq.Workplane("front").box(2, 2, 2).faces("+Z").shell(0.05)

replay(result)

Done, using side car 'CadQuery'


## Ex018 Making Lofts

In [22]:
# Create a lofted section between a rectangle and a circular section.
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the named plane orientation "front" to define the workplane, meaning
#     that the positive Z direction is "up", and the negative Z direction
#     is "down".
# 2.  Creates a plain box to base future geometry on with the box() function.
# 3.  Selects the top-most Z face of the box.
# 4.  Draws a 2D circle at the center of the the top-most face of the box.
# 5.  Creates a workplane 3 mm above the face the circle was drawn on.
# 6.  Draws a 2D circle on the new, offset workplane.
# 7.  Creates a loft between the circle and the rectangle.
result = cq.Workplane("front").box(4.0, 4.0, 0.25).faces(">Z") \
                              .circle(1.5).workplane(offset=3.0) \
                              .rect(0.75, 0.5) \
                              .loft(combine=True)

replay(result)

Done, using side car 'CadQuery'


## Ex019 Counter Sunk Holes

In [23]:
# Create a plate with 4 counter-sunk holes in it.
# 1.  Establishes a workplane using an XY object instead of a named plane.
# 2.  Creates a plain box to base future geometry on with the box() function.
# 3.  Selects the top-most face of the box and established a workplane on that.
# 4.  Draws a for-construction rectangle on the workplane which only exists for
#     placing other geometry.
# 5.  Selects the corner vertices of the rectangle and places a counter-sink
#     hole, using each vertex as the center of a hole using the cskHole()
#     function.
# 5a. When the depth of the counter-sink hole is set to None, the hole will be
#     cut through.
result = cq.Workplane(cq.Plane.XY()).box(4, 2, 0.5).faces(">Z") \
                 .workplane().rect(3.5, 1.5, forConstruction=True) \
                 .vertices().cskHole(0.125, 0.25, 82.0, depth=None)

replay(result)

Done, using side car 'CadQuery'


## Ex020 Rounding Corners with Fillets

In [24]:
# Create a plate with 4 rounded corners in the Z-axis.
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the X and Y origins to define the workplane, meaning that the
#     positive Z direction is "up", and the negative Z direction is "down".
# 2.  Creates a plain box to base future geometry on with the box() function.
# 3.  Selects all edges that are parallel to the Z axis.
# 4.  Creates fillets on each of the selected edges with the specified radius.
result = cq.Workplane("XY").box(3, 3, 0.5).edges("|Z").fillet(0.125)

replay(result)

Done, using side car 'CadQuery'


## Ex021 Splitting an Object

In [25]:
# Create a simple block with a hole through it that we can split.
# 1.  Establishes a workplane that an object can be built on.
# 1a. Uses the X and Y origins to define the workplane, meaning that the
#     positive Z direction is "up", and the negative Z direction is "down".
# 2.  Creates a plain box to base future geometry on with the box() function.
# 3.  Selects the top-most face of the box and establishes a workplane on it
#     that new geometry can be built on.
# 4.  Draws a 2D circle on the new workplane and then uses it to cut a hole
#     all the way through the box.
c = cq.Workplane("XY").box(1, 1, 1).faces(">Z").workplane() \
                      .circle(0.25).cutThruAll()

# 5.  Selects the face furthest away from the origin in the +Y axis direction.
# 6.  Creates an offset workplane that is set in the center of the object.
# 6a. One possible improvement to this script would be to make the dimensions
#     of the box variables, and then divide the Y-axis dimension by 2.0 and
#     use that to create the offset workplane.
# 7.  Uses the embedded workplane to split the object, keeping only the "top"
#     portion.
result = c.faces(">Y").workplane(-0.5).split(keepTop=True)

replay(result)

Done, using side car 'CadQuery'


## Ex022 Revolution

In [26]:
# The dimensions of the model. These can be modified rather than changing the
# shape's code directly.
rectangle_width = 10.0
rectangle_length = 10.0
angle_degrees = 360.0

# Revolve a cylinder from a rectangle
# Switch comments around in this section to try the revolve operation with different parameters
result = cq.Workplane("XY").rect(rectangle_width, rectangle_length, False).revolve()

replay(result)

Done, using side car 'CadQuery'


In [28]:
result = cq.Workplane("XY").rect(rectangle_width, rectangle_length, False).revolve(angle_degrees)

replay(result)

Done, using side car 'CadQuery'


In [29]:
result = cq.Workplane("XY").rect(rectangle_width, rectangle_length).revolve(angle_degrees,(-5,-5))

replay(result)

Done, using side car 'CadQuery'


In [30]:
result = cq.Workplane("XY").rect(rectangle_width, rectangle_length).revolve(angle_degrees,(-5, -5),(-5, 5))

replay(result)

Done, using side car 'CadQuery'


In [32]:
result = cq.Workplane("XY").rect(rectangle_width, rectangle_length).revolve(angle_degrees,(-5,-5),(-5,5), False)

replay(result)

Done, using side car 'CadQuery'


In [34]:
# Revolve a donut with square walls
result = cq.Workplane("XY").rect(rectangle_width, rectangle_length, True).revolve(angle_degrees, (20, 0), (20, 10))

replay(result)

Done, using side car 'CadQuery'


## Ex023 Sweep

In [35]:
# Points we will use to create spline and polyline paths to sweep over
pts = [
    (0, 1),
    (1, 2),
    (2, 4)
]

# Spline path generated from our list of points (tuples)
path = cq.Workplane("XZ").spline(pts)

# Sweep a circle with a diameter of 1.0 units along the spline path we just created
defaultSweep = cq.Workplane("XY").circle(1.0).sweep(path)

# Sweep defaults to making a solid and not generating a Frenet solid. Setting Frenet to True helps prevent creep in
# the orientation of the profile as it is being swept
frenetShell = cq.Workplane("XY").circle(1.0).sweep(path, makeSolid=True, isFrenet=True)

# We can sweep shapes other than circles
defaultRect = cq.Workplane("XY").rect(1.0, 1.0).sweep(path)

# Switch to a polyline path, but have it use the same points as the spline
path = cq.Workplane("XZ").polyline(pts)

# Using a polyline path leads to the resulting solid having segments rather than a single swept outer face
plineSweep = cq.Workplane("XY").circle(1.0).sweep(path)

# Switch to an arc for the path
path = cq.Workplane("XZ").threePointArc((1.0, 1.5), (0.0, 1.0))

# Use a smaller circle section so that the resulting solid looks a little nicer
arcSweep = cq.Workplane("XY").circle(0.5).sweep(path)

In [36]:
replay(defaultSweep)

Done, using side car 'CadQuery'


In [45]:
replay(frenetShell.translate((5, 0, 0)))

Done, using side car 'CadQuery'


In [42]:
replay(defaultRect.translate((10, 0, 0)))

Done, using side car 'CadQuery'


In [43]:
replay(plineSweep.translate((15, 0, 0)))

Done, using side car 'CadQuery'


In [44]:
replay(arcSweep.translate((20, 0, 0)))

Done, using side car 'CadQuery'


## Ex024 Sweep Along List Of Wires

In [46]:
# X axis line length 20.0
path = cq.Workplane("XZ").moveTo(-10, 0).lineTo(10, 0)

# Sweep a circle from diameter 2.0 to diameter 1.0 to diameter 2.0 along X axis length 10.0 + 10.0
defaultSweep = cq.Workplane("YZ").workplane(offset=-10.0).circle(2.0). \
    workplane(offset=10.0).circle(1.0). \
    workplane(offset=10.0).circle(2.0).sweep(path, sweepAlongWires=True)

# We can sweep thrue different shapes
recttocircleSweep = cq.Workplane("YZ").workplane(offset=-10.0).rect(2.0, 2.0). \
    workplane(offset=8.0).circle(1.0).workplane(offset=4.0).circle(1.0). \
    workplane(offset=8.0).rect(2.0, 2.0).sweep(path, sweepAlongWires=True)

circletorectSweep = cq.Workplane("YZ").workplane(offset=-10.0).circle(1.0). \
    workplane(offset=7.0).rect(2.0, 2.0).workplane(offset=6.0).rect(2.0, 2.0). \
    workplane(offset=7.0).circle(1.0).sweep(path, sweepAlongWires=True)


# Placement of the Shape is important otherwise could produce unexpected shape
specialSweep = cq.Workplane("YZ").circle(1.0).workplane(offset=10.0).rect(2.0, 2.0). \
    sweep(path, sweepAlongWires=True)

# Switch to an arc for the path : line l=5.0 then half circle r=4.0 then line l=5.0
path = cq.Workplane("XZ").moveTo(-5, 4).lineTo(0, 4). \
        threePointArc((4, 0), (0, -4)).lineTo(-5, -4)

# Placement of different shapes should follow the path
# cylinder r=1.5 along first line
# then sweep allong arc from r=1.5 to r=1.0
# then cylinder r=1.0 along last line
arcSweep = cq.Workplane("YZ").workplane(offset=-5).moveTo(0, 4).circle(1.5). \
    workplane(offset=5).circle(1.5). \
    moveTo(0, -8).circle(1.0). \
    workplane(offset=-5).circle(1.0). \
    sweep(path, sweepAlongWires=True)

In [47]:
replay(defaultSweep)

Done, using side car 'CadQuery'


In [48]:
replay(circletorectSweep.translate((0, 5, 0)))

Done, using side car 'CadQuery'


In [49]:
replay(recttocircleSweep.translate((0, 10, 0)))

Done, using side car 'CadQuery'


In [50]:
replay(specialSweep.translate((0, 15, 0)))

Done, using side car 'CadQuery'


In [ ]:
replay(arcSweep.translate((0, -5, 0)))

## Ex100 Lego Brick

In [8]:
#####
# Inputs
######
lbumps = 4       # number of bumps long
wbumps = 2        # number of bumps wide
thin = True  # True for thin, False for thick

#
# Lego Brick Constants-- these make a lego brick a lego :)
#
pitch = 8.0
clearance = 0.1
bumpDiam = 4.8
bumpHeight = 1.8
if thin:
    height = 3.2
else:
    height = 9.6

t = (pitch - (2 * clearance) - bumpDiam) / 2.0
postDiam = pitch - t  # works out to 6.5
total_length = lbumps*pitch - 2.0*clearance
total_width = wbumps*pitch - 2.0*clearance

# make the base
s = cq.Workplane("XY").box(total_length, total_width, height)

# shell inwards not outwards
s = s.faces("<Z").shell(-1.0 * t)

# make the bumps on the top
s = s.faces(">Z").workplane(). \
    rarray(pitch, pitch, lbumps, wbumps, True).circle(bumpDiam / 2.0) \
    .extrude(bumpHeight)

# add posts on the bottom. posts are different diameter depending on geometry
# solid studs for 1 bump, tubes for multiple, none for 1x1
tmp = s.faces("<Z").workplane(invert=True)

if lbumps > 1 and wbumps > 1:
    tmp = tmp.rarray(pitch, pitch, lbumps - 1, wbumps - 1, center=True). \
        circle(postDiam / 2.0).circle(bumpDiam / 2.0).extrude(height - t)
elif lbumps > 1:
    tmp = tmp.rarray(pitch, pitch, lbumps - 1, 1, center=True). \
        circle(t).extrude(height - t)
elif wbumps > 1:
    tmp = tmp.rarray(pitch, pitch, 1, wbumps - 1, center=True). \
        circle(t).extrude(height - t)
else:
    tmp = s
    
replay(tmp)

Done, using side car 'CadQuery'
